# KPIs Perfil

In [4]:
import re 
import subprocess
import json
from typing import List, Dict, Tuple
import sys
import os 
from dotenv import load_dotenv

sys.path.insert(0, "../../src")
load_dotenv()
OPENAI_TOKEN = os.getenv("OPENAI_TOKEN")

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_TOKEN)

def call_gpt(prompt: str) -> str:
    response = client.responses.create(
        model="gpt-5-nano",
        input=prompt,
    )
    return response.output_text

In [8]:
def extraer_texto_vendedor(transcript: List[Dict[str, str]]) -> str:
    """Extrae todo el texto del vendedor de la transcripción"""
    return " ".join([entry["text"] for entry in transcript if entry["speaker"] == "vendedor"])

## 1. Prospección

In [21]:
def evaluar_prospeccion(transcript) -> Tuple[int, str]:
  
    prompt = f"""
Analiza la siguiente transcripción de una llamada de ventas y evalúa la PROSPECCIÓN del vendedor según esta rúbrica:

**ALTA (3 puntos)**: El vendedor muestra investigación previa específica sobre el cliente, empresa o situación. Ejemplos: menciona LinkedIn, artículos publicados, eventos recientes, ascensos, situación específica de la empresa.

**MEDIA (2 puntos)**: El vendedor hace preguntas generales de descubrimiento sin personalización. Ejemplos: "cuéntame sobre tu empresa", "qué retos enfrentan", "qué soluciones usan".

**BAJA (1 punto)**: El vendedor va directo al pitch sin fase de descubrimiento. Ejemplos: "te llamo para presentar nuestro producto", "permíteme explicarte las ventajas".

TRANSCRIPCIÓN:
{transcript}

Responde ÚNICAMENTE devolviendo:
- puntuacion: 1, 2 o 3
- justificacion: Explicación específica con ejemplos de la transcripción
"""
    output = call_gpt(prompt)
    
    return output
    

## 2. Empatía

In [24]:
def evaluar_empatia(transcript) -> Tuple[int, str]:
  
    prompt = f"""
Analiza la siguiente transcripción y evalúa la EMPATÍA del vendedor:

**ALTA (3 puntos)**: Valida activamente emociones del cliente, demuestra escucha genuina, parafrasea lo dicho, usa frases como "entiendo perfectamente tu preocupación", "si te entiendo bien", "te agradezco que seas sincero".

**MEDIA (2 puntos)**: Validación genérica o superficial. Respuestas como "ok, entiendo", "claro", "sí, es importante" pero sin profundizar.

**BAJA (1 punto)**: Ignora preocupaciones, interrumpe frecuentemente, descarta objeciones con frases como "el precio no es problema", "sí, pero déjame decirte", "volviendo al tema".


TRANSCRIPCIÓN:
{transcript}

Presta especial atención a:
- ¿El vendedor interrumpe al cliente?
- ¿Valida las preocupaciones antes de responder?
- ¿Usa un lenguaje empático y de comprensión?

Responde ÚNICAMENTE devolviendo:
- puntuacion: 1, 2 o 3
- justificacion: Explicación específica con ejemplos de la transcripción
"""
    text = call_gpt(prompt)
    return text
    

## 3. Dominio técnico

In [26]:
def evaluar_dominio_tecnico(transcript) -> Tuple[int, str]:
  
    prompt = f"""
Analiza la siguiente transcripción y evalúa el DOMINIO TÉCNICO del vendedor:

**ALTA (3 puntos)**: Traduce características técnicas en beneficios específicos para el cliente. Ejemplos: "nuestra batería dura 12 horas, lo que significa que tus clientes no tendrán que recargarla", "la tecnología X reduce el procesamiento en 30%, permitiéndote ahorrar costes".

**MEDIA (2 puntos)**: Menciona características del producto pero sin conectarlas claramente con beneficios. Ejemplos: "tiene batería de 12 horas", "es resistente al agua", "utiliza tecnología X".

**BAJA (1 punto)**: No menciona características específicas o no puede responder preguntas técnicas. Ejemplos: "es muy bueno", "funciona bien", "déjame preguntar a mi superior".

TRANSCRIPCIÓN:
{transcript}

Evalúa si el vendedor:
- Conoce las características técnicas del producto
- Las conecta con beneficios específicos
- Responde con confianza a preguntas técnicas

Responde ÚNICAMENTE devolviendo:
- puntuacion: 1, 2 o 3
- justificacion: Explicación específica con ejemplos de la transcripción
"""
    output = call_gpt(prompt)
    
    return output
    

## 4. Negociación

In [27]:
def evaluar_negociacion(transcript) -> Tuple[int, str]:
  
    prompt = f"""
Analiza la siguiente transcripción y evalúa la NEGOCIACIÓN del vendedor:

**ALTA (3 puntos)**: Aborda objeciones justificando el valor, es firme pero flexible en condiciones, propone próximos pasos claros. Ejemplos: "entiendo que el precio es superior, pero es una inversión que se justifica por el ahorro", "podríamos ajustar las condiciones de pago", "el siguiente paso sería programar una demo".

**MEDIA (2 puntos)**: Evita objeciones o cede demasiado pronto, ofrece descuentos sin que se los pidan. Ejemplos: "el precio es negociable", "te mando la propuesta otra vez", "veo si puedo ofrecerte descuento".

**BAJA (1 punto)**: Se rinde ante la primera objeción, no intenta cerrar, deja decisión en manos del cliente. Ejemplos: "avísame si te interesa", "llámame si te decides", "no te preocupes, lo entiendo".

TRANSCRIPCIÓN:
{transcript}

Evalúa:
- ¿Cómo maneja las objeciones?
- ¿Justifica el valor o solo ofrece descuentos?
- ¿Propone próximos pasos concretos?

Responde ÚNICAMENTE devolviendo:
- puntuacion: 1, 2 o 3
- justificacion: Explicación específica con ejemplos de la transcripción

"""
    output = call_gpt(prompt)
    
    return output
    

## 5. Resiliencia

In [34]:
def evaluar_resiliencia(transcript) -> Tuple[int, str]:

    prompt = f"""
Analiza la siguiente transcripción y evalúa la RESILIENCIA del vendedor:

**ALTA (3 puntos)**: Mantiene tono positivo y enérgico durante toda la llamada, se recupera rápidamente de objeciones, usa lenguaje constructivo. Ejemplos: "aprecio tu franqueza, permíteme explicarte", mantiene energía constante incluso tras objeciones de precio.

**MEDIA (2 puntos)**: El tono puede bajar ligeramente tras objeciones, pequeñas pausas o disminución de energía, pero se recupera.

**BAJA (1 punto)**: Muestra frustración, tono negativo o pasivo, se rinde fácilmente. Ejemplos: suspiros audibles, "entiendo, si no te interesa no pasa nada" con tono monótono.

TRANSCRIPCIÓN:
{transcript}

Presta atención a:
- ¿Mantiene un tono consistente durante toda la llamada?
- ¿Cómo responde a objeciones o rechazos?
- ¿Se detectan cambios de energía o frustración?
- ¿Termina la llamada con próximos pasos o se rinde?

Responde ÚNICAMENTE devolviendo:
- puntuacion: 1, 2 o 3
- justificacion: Explicación específica con ejemplos de la transcripción

Limita el output a menos de 20 palabras
"""
    output = call_gpt(prompt)
    
    return output

    

# Funcionalidad

In [15]:
transcript_bueno = [
    {"speaker": "vendedor", "text": "Buenas tardes Juan, ¿cómo estás? Te llamo a ver para seguir con el tema de la reunión de la semana pasada. ¿Qué tal te ha ido este la semana? ¿Alguna novedad?"},
    {"speaker": "cliente", "text": "Buenas tardes Pedro, pues la verdad es que bastante bien, gracias por preguntar. Y sí, justo pensaba en vuestra propuesta. La he estado revisando con mi equipo y tenemos algunas preguntas que me gustaría que me aclaraces para poder tomar una decisión."},
    {"speaker": "vendedor", "text": "Ah, perfecto. Ehm, de eso se trata. ¿Qué dudas tienes este, a ver en concreto? Y bueno, así las resolvemos ahora mismo. Estoy aquí para lo que necesites."},
    {"speaker": "cliente", "text": "Mira, la principal es sobre el coste. Entendemos que vuestros auriculares son de gama alta, pero el presupuesto que nos habéis pasado es un poco superior al que teníamos planeado. Necesito entender mejor, ¿eh?, por qué la inversión es tan elevada."},
    {"speaker": "vendedor", "text": "Entiendo perfectamente. Y bueno, el precio es un factor importante, claro. Y quiero que te sientas seguro con tu decisión. Pero, este, a ver, permíteme explicarte de nuevo lo que hace que nuestros auriculares sean diferentes y a la vez que justifique esa inversión. Verás, no solo estás comprando un producto. Este, estás adquiriendo una serie de ventajas que van a impactar directamente en la experiencia del usuario, en la reputación de vuestra marca y en el valor añadido que ofrecéis."},
    {"speaker": "cliente", "text": "¿A qué te refieres exactamente? ¿Puede ser un poco más específico con las ventajas?"},
    {"speaker": "vendedor", "text": "A ver, como te comenté la semana pasada, una de las grandes diferencias es el volumen de escucha. Y bueno, es un detalle que a mi nudo se pasa por alto. Pero, este, a ver, es fundamental para la satisfacción del cliente. Nuestros auriculares, este, tienen un nivel de volumen que es un 20% más alto que la media del mercado, sin perder la calidad del sonido ni distorsiones. Esto es crucial para los clientes que busquen una experiencia inmersiva. Y este, y bueno, que necesitan un sonido claro en entornos ruidosos, como en el transporte público o en la caña. ¿Te imaginas a alguien intentando escuchar música en una cafetería ruidosa? Con nuestros auriculares, no tendría ningún problema."},
    {"speaker": "cliente", "text": "Eso sí que me parece interesante. No había considerado ese punto con tanta profundidad. Es verdad que muchos clientes se quejan de que el volumen máximo no es suficiente. ¿Qué otras cosas destacarías? Porque si bien ese punto es importante, necesito más argumentos para justificar el precio."},
    {"speaker": "vendedor", "text": "Pues mira, también está la resistencia al agua. Y este punto es revolucionario. Nuestros auriculares son subacuáticos. No solo son resistentes a salpicadura o al sudor. Este, a ver, sino que se pueden sumergir completamente a una profundidad de hasta un metro durante 30 minutos. Y bueno, esto abre un mercado completamente nuevo para vuestros clientes. Piensa en deportistas, nadadores, que quieren escuchar música mientras entrenan. Gente que hace ejercicio al aire libre bajo la lluvia. Y bueno, también usuarios que simplemente quieren la tranquilidad de saber que sus auriculares no se van a estropear si les cae un vaso de agua encima."},
    {"speaker": "cliente", "text": "Uff, eso es un valor añadido. No me digas más. Es una característica que nos diferenciaría completamente de la competencia. Y qué me dices de la batería? Siempre tengo un poco de miedo de que se descarguen en el momento más inoportuno. Es una de las quejas más comunes entre los usuarios de auriculares inalámbricos."},
    {"speaker": "vendedor", "text": "Y bueno, ese es otra gran ventaja, Juan. La duración de la batería. Nuestros auriculares ofrecen hasta 12 horas de reproducción continua con una sola carga. A ver, eso es casi el doble que la mayoría de nuestros competidores. Imagina un usuario que viaja en avión, por ejemplo, o que tiene un día largo en el trabajo. No tendría que preocuparse de cargar los auriculares a cada rato. Eso es un alivio para ellos y bueno, se traduce en una mejor experiencia de uso."},
    {"speaker": "cliente", "text": "La verdad, Pedro, es que ahora que lo dices así con más detalle, le veo mucho más sentido al precio. El volumen, la resistencia al agua y la duración de la batería son tres puntos muy fuertes. ¿Qué garantía ofrecen para el producto?"},
    {"speaker": "vendedor", "text": "A ver, ofrecemos una garantía de dos años contra cualquier defecto de fabricación. Y bueno, también tenemos un servicio de atención al cliente, que está disponible 24-7 para cualquier problema que pueda surgir. Queremos que tanto vosotros como vuestros clientes estén completamente satisfechos con la compra."},
    {"speaker": "cliente", "text": "Eso me da mucha tranquilidad. Bien, entiendo el valor del producto. Ahora, volviendo al tema del precio, ¿hay alguna posibilidad de ajustar las condiciones de pago, ofrecen algún tipo de descuento por volumen? Si compramos una cantidad significativa, ¿claro?"},
    {"speaker": "vendedor", "text": "Mmm... A ver, lo podemos negociar, claro. Y bueno, me gustaría entender mejor vuestras necesidades. ¿Qué cantidad de unidades estarías, este, a ver, pensando en adquirir inicialmente? Y con esa información podríamos buscar una opción que se ajuste a vuestras necesidades de presupuesto y de flujo de caja."},
    {"speaker": "cliente", "text": "¿Estaríamos pensando en un pedido inicial de unas 5.000 unidades? Es un número importante para nosotros."},
    {"speaker": "vendedor", "text": "5.000 unidades, ¿entendido? Y bueno, con un volumen así, definitivamente podríamos aplicar un descuento por volumen. Y a ver, también podríamos, este, revisar las condiciones de pago. ¿Qué tal si lo hablamos la semana que viene? Así, preparo una propuesta con esos ajustes."},
    {"speaker": "cliente", "text": "Me parece perfecto. Te llamo yo el martes por la mañana, ¿te parece bien?"},
    {"speaker": "vendedor", "text": "Sí, claro. Quedamos así entonces. Y bueno, un placer hablar contigo, Juan. Espero que esta nueva información te ayude a tomar la mejor decisión."},
    {"speaker": "cliente", "text": "Igualmente, Pedro. Y gracias por la paciencia y por aclarar todas mis dudas. Hasta el martes."}
]
transcript_cortante = [
    {"speaker": "vendedor", "text": "Buenas tardes Juan, ¿qué tal? ¿Cómo estás? Te llamo a ver para seguir con el tema de la reunión de la semana pasada, que... quedamos en que te llamaría para ver qué te había parecido la propuesta. ¿Qué tal te ha ido la semana? ¿Alguna novedad? Me imagino que sí, claro. Y bueno, supongo que habrás visto los auriculares y todo lo que te mandé, ¿verdad? Y este... a ver, quiero saber qué opinas, porque la verdad es que estamos muy contentos con el producto que tenemos entre manos y bueno, creemos que tiene un potencial increíble para vuestra marca. Ya te comenté que la experiencia de usuario que ofrecemos es superior a todo lo que hay en el mercado, este... a ver, en cuanto a calidad y prestaciones. Y... ¿y bueno, qué me dices?"},
    {"speaker": "cliente", "text": "Buenas tardes Pedro. Pues la verdad es que..."},
    {"speaker": "vendedor", "text": "Sí, claro, dime, dime, te escucho. Seguro que tienes alguna pregunta sobre el coste, ¿verdad? A ver, es lo más habitual, porque entiendo que el precio puede parecer un poco elevado a primera vista, pero... y bueno, como te dije, es una inversión que realmente merece la pena. No estamos hablando de unos auriculares más. A ver, estamos hablando de un producto de muy alta gama, con unas características que no vas a encontrar en ningún otro sitio. Me imagino que, este, a ver, lo habrás pensado y... y bueno, seguro que tienes alguna duda concreta. Yo, la verdad, estoy convencido de que os va a encantar. Y bueno, de que vuestros clientes van a estar fascinados con la calidad del sonido, que es, este, a ver, es espectacular."},
    {"speaker": "cliente", "text": "Sí, de hecho el precio..."},
    {"speaker": "vendedor", "text": "Claro, el precio. Y bueno, déjame que te recuerde los puntos clave. A ver, la gran diferencia es el volumen de escucha. Te lo comenté, pero, este, a ver, quiero que te quede claro. Tenemos un 20% más de volumen que la competencia. Es, este, a ver, es una barbaridad. Y bueno, eso permite que el usuario escuche su música, sus podcasts, lo que sea. Este, a ver, en cualquier entorno, sin que le moleste el ruido de fuera. Y sin perder, este, a ver, ni una pise de calidad. Y bueno, eso es una ventaja competitiva muy importante que, este, a ver, os va a diferenciar mucho. Además, este, a ver, si ves cómo está el mercado ahora, todo mundo busca algo que le haga destacar. Y bueno, la calidad de nuestros auriculares es, a ver, es inigualable. Y bueno, es un valor seguro para vosotros. Además, este, a ver, también está el tema de la resistencia al agua. Nuestros auriculares son subacuáticos. Sí, sí, como lo oyes, se pueden sumergir. Y bueno, esto es una maravilla para, a ver, para los deportistas. Y para la gente que vive en zonas con mucha lluvia. Y bueno, para, este, a ver, para cualquier persona que, que quiera que tenga la tranquilidad de que sus auriculares no se van a estropear con este, con un poco de agua. Y bueno, la batería también es un, este, a ver, un punto muy fuerte. Tenemos 12 horas de duración. Eso es el doble de la media. O sea, este, a ver, es que no hay comparación. Es, a ver, la mejor opción del mercado. Y bueno, ¿qué te parece? ¿Verdad que es increíble?"},
    {"speaker": "cliente", "text": "Sí, la verdad me parece muy bien. Pero tengo una pregunta sobre el..."},
    {"speaker": "vendedor", "text": "Seguro que sobre las condiciones de pago. Y este, a ver, no te preocupas por eso, lo podemos negociar. Y bueno, estamos abiertos a, este, hablar de los plazos y de los, a ver, de los volúmenes de compra. Sí, este, a ver, si te interesa un pedido grande, podemos, este, a ver, hacer un descuento especial. Y bueno, la idea es que todos salgamos ganando. Y bueno, ¿qué te parece? ¿Verdad que es una propuesta muy, este, muy atractiva? Ya te digo, ¿eh? Con un pedido grande, podríamos, este, a ver, ofrecer un precio por unidad que se ajuste más a vuestro presupuesto inicial. Y bueno, te aseguro que, este, la calidad del producto y, a ver, la satisfacción del cliente compensa, eh, con creces esa inversión. Y, y bueno, además, este, a ver, la marca de auriculares que ofreces dice mucho de ti. Y bueno, ofrecer unos auriculares con estas prestaciones es, a ver, es proyectar una imagen de marca de, de alta calidad y, este, de innovación."},
    {"speaker": "cliente", "text": "Sí"},
    {"speaker": "vendedor", "text": " Me alegro de que lo vías así. Y bueno, mira, para no, eh, para no alargarnos más, ¿qué te parece si te envió la propuesta de nuevo? Pero, este, a ver, con un pequeño resumen de las ventajas competitivas, este, a ver, en Negrita, para que se lo puedas enseñar a tu equipo y, y bueno, que vean que la inversión está más que justificada. Y este, a ver, te aseguro que no os vais a arrepentir. Y, y bueno, podríamos, eh, concertar una reunión con tu equipo si quieres, para, este, a ver, para resolver cualquier duda que tengan. Y, y bueno, si lo ven con más detalle."},
    {"speaker": "cliente", "text": "Me parece bien, pero me gustaría..."}, 
    {"speaker": "vendedor", "text": "Perfecto, Juan. Y bueno, quedemos así. Te lo envío ahora mismo y, este, a ver, me llama si tienes alguna otra pregunta y, bueno, hablamos la semana que viene para cerrar los, los detalles. Y este, a ver, seguro que todo sale bien. Y, y bueno, un placer."},
    {"speaker": "cliente", "text": "Igualmente, Pedro. Hasta el martes."}
]

In [22]:
evaluar_prospeccion(transcript_bueno)

'puntuacion: 2\njustificacion: Evaluación de PROSPECCIÓN: MEDIA (2 puntos). Razones:  \n- Se observa continuidad con una reunión previa y preguntas para aclarar dudas, pero no hay investigación específica sobre el cliente o la empresa (no se mencionan LinkedIn, artículos, eventos, ascensos o una situación concreta de la empresa).  \n- Hay preguntas de descubrimiento, pero mayormente generales: "¿Qué dudas tienes?" y "¿Qué cantidad de unidades estarías pensando en adquirir inicialmente?", sin personalización profunda hacia el negocio del cliente.  \n- Aunque se explican ventajas del producto (volumen, resistencia al agua, batería) y se intenta adaptar la oferta (volumen de 5.000 unidades) y las condiciones de pago, la prospección no muestra un análisis detallado de necesidades específicas del cliente ni investigación previa marcada.'

In [25]:
evaluar_empatia(transcript_cortante)

'puntuacion: 2\njustificacion: El vendedor valida algunas preocupaciones y demuestra escucha básica: dice "Seguro que tienes alguna pregunta sobre el coste" y "entiendo que el precio puede parecer un poco elevado a primera vista", lo que indica reconocimiento de la preocupación del cliente y disponibilidad para escuchar. También dice "te escucho" y evita interrumpir de forma abrupta. Sin embargo, no utiliza un lenguaje empático muy profundo ni parafrasea emociones del cliente (no dice frases como "entiendo perfectamente tu preocupación" ni "si te entiendo bien"), y tiende a centrarse en argumentos de venta, por lo que no alcanza empatía alta.'

In [29]:
evaluar_negociacion(transcript_bueno)

'puntuacion: 3\njustificacion: El vendedor maneja objetivamente las objeciones y no se limita a bajar precios. Aborda la preocupación de costo explicando el valor agregado: "permíteme explicarte de nuevo lo que hace que nuestros auriculares sean diferentes y a la vez que justifique esa inversión" y enumera ventajas concretas como "volumen de escucha ... un 20% más alto", "resistencia al agua ... subacuáticos" y "la duración de la batería. Nuestros auriculares ofrecen hasta 12 horas de reproducción" además de la garantía de dos años y servicio 24/7. No se queda en justificar el valor; también propone condiciones comerciales alternas: "podríamos aplicar un descuento por volumen" y "revisar las condiciones de pago". Propone próximos pasos claros: calcular cantidad inicial (5.000 unidades) para ajustar precio, "¿Qué tal si lo hablamos la semana que viene? Así, preparo una propuesta con esos ajustes", y acuerda una llamada (cliente: "Te llamo yo el martes por la mañana"). En conjunto, equil

In [30]:
evaluar_negociacion(transcript_cortante)

'puntuacion: 3\njustificacion: La venta maneja objeciones justificando claramente el valor, no solo oferecendo precio. El vendedor presenta argumentos de alto valor: "no estamos hablando de unos auriculares más. A ver, estamos hablando de un producto de muy alta gama, con unas características que no vas a encontrar en ningún otro sitio." y detalla beneficios específicos: "Tenemos un 20% más de volumen", "12 horas de duración", "resistencia al agua" y destaca cómo estas características justifican la inversión: "una inversión que realmente merece la pena" y que "la calidad del producto y, a ver, la satisfacción del cliente compensa, con creces esa inversión." Además, reconoce la posible objeción de precio y ofrece flexibilización condicional: "podemos negociar las condiciones de pago" y "con un pedido grande, podríamos... ofrecer un descuento especial" (descuento condicionado a volumen). No se limita a descuentos gratuitos; argumenta el valor y las condiciones para acercar la oferta. Pro

In [31]:
evaluar_empatia(transcript_cortante)

'puntuacion: 2\njustificacion: No interrumpe al cliente: cuando el cliente habla, el vendedor dice “Sí, claro, dime, dime, te escucho” y continúa tras escuchar. Valida preocupaciones antes de responder: reconoce la posible duda de coste con frases como “Seguro que tienes alguna pregunta sobre el coste” y “entiendo que el precio puede parecer un poco elevado a primera vista” y además contextualiza la inversión. Usa lenguaje empático: emplea “te escucho”, “entiendo”, “Me alegro de que lo vías así” y ofrece facilidades (descuentos por pedido grande, resumen para el equipo) para facilitar la decisión. Aunque no parafrasea extensamente ni usa frases explícitamente como “entiendo perfectamente tu preocupación”, su tono y acciones muestran comprensión moderada.'

In [35]:
evaluar_resiliencia(transcript_cortante)

'puntuacion: 3\njustificacion: Tono constante y positivo; maneja objeciones de precio con argumentos y ofertas; termina con próximos pasos.'